**Feistel 网络加密解密**

In [94]:
def feistel_encrypt(plain_text, num_rounds, round_function, key):
    L, R = plain_text[:len(plain_text)//2], plain_text[len(plain_text)//2:]
    for i in range(num_rounds):
        F = round_function(R, key[100*i:100*(i+1)])
        new_R = round_function(L, F)
        L, R = R, new_R
    return L + R

def feistel_decrypt(cipher_text, num_rounds, round_function, key):
    L, R = cipher_text[:len(cipher_text)//2], cipher_text[len(cipher_text)//2:]
    for i in range(num_rounds):
        F = round_function(L, key[100*(num_rounds-i-1):100*(num_rounds-i)])
        new_L = round_function(R, F)
        L, R = new_L, L
    return L + R
    
# 传入两个字符串参数，在函数内部将其转换为二进制字符串，然后进行异或运算，最后返回字符串结果
def round_function(data, key):
    result = ""
    for i in range(len(data)):
        result += str(int(data[i]) ^ int(key[i]))
    return result

def str_to_binary(s):
    return ''.join(format(ord(c), '08b') for c in s)

def binary_to_str(b):
    return ''.join(chr(int(b[i:i+8], 2)) for i in range(0, len(b), 8) if i+8 <= len(b))


**验证字符串->十进制->二进制->字符串 是否正确**

In [66]:
# 明文
plain_text = "CQUINFORMATIONSECURITYEXP"
plain_text_binary = str_to_binary(plain_text)
print(len(plain_text))
print(plain_text_binary)
print(len(plain_text_binary))
print(type(plain_text_binary))
text = binary_to_str(plain_text_binary)
print(text)

25
01000011010100010101010101001001010011100100011001001111010100100100110101000001010101000100100101001111010011100101001101000101010000110101010101010010010010010101010001011001010001010101100001010000
200
<class 'str'>
CQUINFORMATIONSECURITYEXP


**设定密钥，考虑到实际明文为200位，设定密钥为100位**

In [79]:
# 定义轮函数使用的密钥
# 由于明文为200位，所以每一轮的密钥长度为100位，共有16轮
import random
key = [random.randint(0, 1) for _ in range(1600)]
# 转为字符串
key_str = ''.join([str(i) for i in key])
print(key_str, len(key_str))
for i in range(16):
    print(key_str[100*i:100*(i+1)])

1001001111001100001100001010000101101001011000000011111110000001000001101110000001010101000100001010111101010101000110101011010001011110001100100101001010010011001110011111110111010110000000000101000000001111111110100000101111010000110101101010010001000010111001100010100000011101111010001101101001111011010111111100010101101101000010111011001110010010000011110011001000001000011000111110011001000100110101100100000101100100100000011101011001011001110011111100111001111111011010000101100010111000110010011000100000111101000011011111100111110001001000101011110100000011010010100011011000101100010001100000011100111010000100101001010000111011001101010000001001110010111100010101011000111000100111000101110110011010101011100010010001101100100110011101011101110011010111010010011011011111111110000100010000100110001011000011001110011111110011111011101101010101011011100101000000001111000110111111001111100100000110111010010000000110011111100100011100000000011101100011001111001100000000111011011100110111

In [73]:
key_test = [random.randint(0, 1) for _ in range(100)]
key_test_str = ''.join([str(i) for i in key_test])
R = plain_text_binary[len(plain_text_binary)//2:]
result = round_function(R, key_test_str)
print(result)
# 将R与密钥异或，需要先将str转为二进制数
R = [int(i) for i in R]
key_test_str = [int(i) for i in key_test_str]
new_R = [a ^ b for a, b in zip(R, key_test_str)]
new_R = ''.join([str(i) for i in new_R])
# print(R, len(R))
# print(key_test_str, len(key_test_str))
print(new_R)

1100010110010100100101110111100001011110110000011100110111110110100101000000111010100001110110100111
1100010110010100100101110111100001011110110000011100110111110110100101000000111010100001110110100111


In [95]:
# 将字符转换为二进制
print("明文：", plain_text)
plain_text_binary = str_to_binary(plain_text)
print("二进制明文：", plain_text_binary)
# 加密
cipher_text_binary = feistel_encrypt(list(map(int, plain_text_binary)), num_rounds=16, round_function=round_function, key=key)
print("二进制密文：", cipher_text_binary)
# 加密结果
cipher_text = binary_to_str(cipher_text_binary)
print("密文：", cipher_text)

明文： CQUINFORMATIONSECURITYEXP
二进制明文： 01000011010100010101010101001001010011100100011001001111010100100100110101000001010101000100100101001111010011100101001101000101010000110101010101010010010010010101010001011001010001010101100001010000
二进制密文： 01100011110001101010110011100010110011101100100111101110000001111010100001110110110101101100011110000000111100101100010110010010011100010010001011100001010010000110010111110101111000010110000110011101
密文： cÆ¬âÎÉî¨vÖÇòÅq"áHeõáa


In [98]:
# 解密
decipher_text_binary = feistel_decrypt(list(map(int, cipher_text_binary)), num_rounds=16, round_function=round_function, key=key)
decipher_text = binary_to_str(decipher_text_binary)

print(f"明文: {plain_text}")
print(f"二进制明文: {plain_text_binary}")
print(f"密文: {cipher_text}")
print(f"二进制密文: {cipher_text_binary}")
print(f"密文解密后: {decipher_text}")
print(f"二进制密文解密后: {decipher_text_binary}")


明文: CQUINFORMATIONSECURITYEXP
二进制明文: 01000011010100010101010101001001010011100100011001001111010100100100110101000001010101000100100101001111010011100101001101000101010000110101010101010010010010010101010001011001010001010101100001010000
密文: cÆ¬âÎÉî¨vÖÇòÅq"áHeõáa
二进制密文: 01100011110001101010110011100010110011101100100111101110000001111010100001110110110101101100011110000000111100101100010110010010011100010010001011100001010010000110010111110101111000010110000110011101
密文解密后: CQUINFORMATIONSECURITYEXP
二进制密文解密后: 01000011010100010101010101001001010011100100011001001111010100100100110101000001010101000100100101001111010011100101001101000101010000110101010101010010010010010101010001011001010001010101100001010000
